In [2]:
import requests
import tensorflow as tf
import json
import base64

In [12]:
# Prepare input JSON payload
def prepare_json(features):
    """
    Converts raw input features into a serialized `tf.Example` format for prediction.

    Args:
        features (dict): Raw input features.

    Returns:
        str: JSON payload ready to be sent to the model endpoint.
    """
    # Define fields expected as integers
    integer_fields = ["person_emp_exp", "credit_score", "loan_status"]

    # Ensure integer fields are cast to int
    for field in integer_fields:
        if field in features:
            features[field] = int(features[field])
    
    # Prepare the serialized example
    feature_spec = {
        key: tf.train.Feature(
            int64_list=tf.train.Int64List(value=[value])
        ) if isinstance(value, int) else
        tf.train.Feature(
            float_list=tf.train.FloatList(value=[value])
        ) if isinstance(value, float) else
        tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[bytes(value, "utf-8")])
        )
        for key, value in features.items()
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    return json.dumps(
        {
            "signature_name": "serving_default",
            "instances": [{"examples": {"b64": base64.b64encode(example).decode()}}]
        }
    )

In [13]:
# Sample Data
data_samples = [
    {
        "person_age": 22.0, "person_gender": "female", "person_education": "Master",
        "person_income": 71948.0, "person_emp_exp": 0, "person_home_ownership": "RENT",
        "loan_amnt": 35000.0, "loan_intent": "PERSONAL", "loan_int_rate": 16.02,
        "loan_percent_income": 0.49, "cb_person_cred_hist_length": 3.0,
        "credit_score": 561, "previous_loan_defaults_on_file": "No"
    },
    {
        "person_age": 21.0, "person_gender": "female", "person_education": "High School",
        "person_income": 12282.0, "person_emp_exp": 0, "person_home_ownership": "OWN",
        "loan_amnt": 1000.0, "loan_intent": "EDUCATION", "loan_int_rate": 11.14,
        "loan_percent_income": 0.08, "cb_person_cred_hist_length": 2.0,
        "credit_score": 504, "previous_loan_defaults_on_file": "Yes"
    },
]

In [17]:
# Endpoint URL
endpoint = "https://mlopsfinalproject-production.up.railway.app/v1/models/loan_approval_model:predict"

# Make requests and print predictions
results = {}

for sample in data_samples:
    response = requests.post(endpoint, data=prepare_json(sample))
    sample_key = json.dumps(sample)  # Convert sample dictionary to a string
    if response.status_code == 200:
        prediction = response.json()["predictions"][0][0]
        results[sample_key] = "Approved" if prediction >= 0.5 else "Rejected"
    else:
        results[sample_key] = f"Error: {response.status_code}, {response.text}"

# Display results
for sample, outcome in results.items():
    print(f"Input: {sample}, Outcome: {outcome}")

Input: {"person_age": 22.0, "person_gender": "female", "person_education": "Master", "person_income": 71948.0, "person_emp_exp": 0, "person_home_ownership": "RENT", "loan_amnt": 35000.0, "loan_intent": "PERSONAL", "loan_int_rate": 16.02, "loan_percent_income": 0.49, "cb_person_cred_hist_length": 3.0, "credit_score": 561, "previous_loan_defaults_on_file": "No"}, Outcome: Approved
Input: {"person_age": 21.0, "person_gender": "female", "person_education": "High School", "person_income": 12282.0, "person_emp_exp": 0, "person_home_ownership": "OWN", "loan_amnt": 1000.0, "loan_intent": "EDUCATION", "loan_int_rate": 11.14, "loan_percent_income": 0.08, "cb_person_cred_hist_length": 2.0, "credit_score": 504, "previous_loan_defaults_on_file": "Yes"}, Outcome: Rejected
